In [1]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from matplotlib import pyplot
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
import re
import joblib

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [19]:
train = pd.read_csv("train_df.csv")
test = pd.read_csv("test_df.csv")
ts_df = pd.read_csv('ts_pred.csv')

In [5]:
pred_date = pd.date_range(start='2021-10-16', end='2021-11-05', freq='1D')
pred_date = pred_date.strftime('%Y-%m-%d').tolist()

forecast_date = pd.date_range(start='2021-11-06', end='2021-12-03', freq='1D')
forecast_date = forecast_date.strftime('%Y-%m-%d').tolist()

pred_forecast_date = pred_date + forecast_date

state_list = ts_df.subregion1_code.unique().tolist()

predictors = ' '.join(train.columns)
mob_predictors = re.findall(r'mobility_\w+', predictors)
searchtrend_predictors = re.findall(r'search_trends_\w+', predictors)
ts_predictors = mob_predictors + searchtrend_predictors

In [9]:
static_latest_raw = pd.DataFrame()
for state in state_list:
    find_df = test[test['subregion1_code'] == state]
    find_df['date'] = pd.to_datetime(find_df['date'])
    date_latest = find_df['date'].max()
    state_row = find_df[find_df['date'] == date_latest]
    static_latest_raw = static_latest_raw.append(state_row)

nj_latest = train[(train['subregion1_code']=='NJ')&(train['date']=='2021-10-04')]
static_latest_raw = static_latest_raw.append(nj_latest)
static_latest = static_latest_raw.drop(["date"]+["cumulative_confirmed"]+ts_predictors, axis = 1)
static_latest = static_latest.reset_index(drop = True)


static_df = pd.DataFrame(np.repeat(static_latest.values, len(pred_forecast_date), axis=0))
static_df.columns = static_latest.columns

static_df = static_df.sort_values(by=['subregion1_code'])

static_df = static_df.reset_index(drop = True)
static_df['date'] = pred_forecast_date * len(state_list)

ValueError: Length of values (2401) does not match length of index (2352)

In [5]:
pred_df = pd.merge(ts_df, static_df, on = ['subregion1_code', 'date'])

pred_df['date'] =pd.to_datetime(pred_df['date'])
pred_df = pred_df.sort_values(by=['subregion1_code', 'date'])
pred_df = pred_df.drop(['type'], axis = 1)

In [6]:
res_df_gbm = pd.DataFrame()

In [7]:
loaded_gbm = joblib.load("GBM_fin.joblib")

In [8]:
#for state in static_latest.subregion1_code.unique():
for state in state_list:
    state_col = [state] * (len(pred_date)+len(forecast_date))
    pred_type = ['prediction'] * (len(pred_date))+ ['forecast'] *len(forecast_date)
    pred_curr = pd.DataFrame(list(zip(state_col, pred_date + forecast_date, pred_type)),
                          columns =['subregion1_code', 'date', 'type'])
    

    s_test = pred_df[pred_df["subregion1_code"]==state]
    
    
    s_test = s_test.drop(["date","subregion1_code"], axis = 1)
    s_X_test = s_test.drop(["new_confirmed"], axis = 1)
    
   # cols = s_X_test.columns[s_X_test.dtypes.eq('object')]
   # s_X_test[cols] = s_X_test[cols].apply(pd.to_numeric, errors='coerce', axis=1)
    
   # s_X_test = s_X_test[cols_when_model_builds]
    
    s_y_pred_new = loaded_gbm.predict(s_X_test)
    
    pred_curr['new_confirmed_pred'] = s_y_pred_new
    
    #create prediction dataframe
    res_df_gbm = res_df_gbm.append(pred_curr)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/Library/Fram

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [76]:
res_df_xgboost.to_csv('xgboost_pred2.csv', index = False)